In [ ]:
def show_images(images, cmap=None):
    cols = 2
    rows = (len(images)+1)//cols
    
    plt.figure(figsize=(15, 12))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        # use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()
    
    
def detect_edges(image, low_threshold , high_threshold):
    return cv2.Canny(image, low_threshold, high_threshold)

def suavizar_gaussian(image,d=5, e= 5):
    return cv2.GaussianBlur(image,(d, e),0)
    #return cv2.GaussianBlur(image, (n1, n2), 0)

def hough_lines(image):
    """
    `image` should be the output of a Canny transform.
    
    Returns hough lines (not the image with lines)
    """ 
    return cv2.HoughLinesP(image, rho=0.1, theta=np.pi/10, threshold=15, minLineLength=9, maxLineGap=4)

def draw_lines(image, lines, color=[255, 0, 0], thickness=2, make_copy=True):
    # the lines returned by cv2.HoughLinesP has the shape (-1, 1, 4)
    if make_copy:
        image = np.copy(image) # don't want to modify the original
    cleaned = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y2-y1) <=1 and abs(x2-x1) >=25 and abs(x2-x1) <= 55:
                cleaned.append((x1,y1,x2,y2))
                cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    print(" No lines detected: ", len(cleaned))
    return image

def select_rgb_white_yellow(image): 
    # white color mask
    lower = np.uint8([120, 120, 120])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower, upper)
    # yellow color mask
    lower = np.uint8([190, 190,   0])
    upper = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

In [ ]:
###Esqueleto



height,width = mask.shape
skel = np.zeros([height,width],dtype=np.uint8)      #[height,width,3]

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
temp_nonzero = np.count_nonzero(mask)

while(np.count_nonzero(mask) != 0 ):
    eroded = cv2.erode(mask,kernel)
    #cv2.imshow("eroded",eroded)   
    temp = cv2.dilate(eroded,kernel)
    #cv2.imshow("dilate",temp)
    temp = cv2.subtract(mask,temp)
    skel = cv2.bitwise_or(skel,temp)
    mask = eroded.copy()

#cv2.imshow("skel",skel)

plt.imshow(skel, cmap='gray')
plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
plt.show() 

In [ ]:
def select_rgb_white_yellow(image): 
    # white color mask
    lower = np.uint8([120, 120, 120])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower, upper)
    # yellow color mask
    lower = np.uint8([190, 190,   0])
    upper = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

result = select_rgb_white_yellow(crop_img)
mask = result
plt.imshow(result)
plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
plt.show() 

In [ ]:
def suavizar_gaussian(image,d=5, e= 5):
    return cv2.GaussianBlur(image,(d, e),0) 

blur_gray = suavizar_gaussian(skel)


plt.imshow(blur_gray, cmap='gray')
plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
plt.show()

In [ ]:
edges = detect_edges(blur_gray, 50 , 150)

plt.imshow(edges, cmap='gray')
plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
plt.show() 

In [ ]:
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 15  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 10  # minimum number of pixels making up a line
max_line_gap = 10  # maximum gap in pixels between connectable line segments
line_image = np.copy(crop_img) * 0  # creating a blank to draw lines on
#min_line_length = 10 
#max_line_gap = 10 

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)

points = []
for line in lines:
    for x1,y1,x2,y2 in line:
        points.append(((x1 + 0.0, y1 + 0.0), (x2 + 0.0, y2 + 0.0)))
        cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
        
        
lines_edges = cv2.addWeighted(crop_img, 0.8, line_image, 1, 0)
plt.imshow(lines_edges , cmap='gray')
plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
# rho=0.1, theta=np.pi/10, threshold=15, minLineLength=9, maxLineGap=4
plt.show() 

In [ ]:
# Detectando Linhas Paralelas
#def identify_blocks(image, lines, make_copy=True):
#    if make_copy:
#        new_image = np.copy(image)
    #Step 1: Create a clean list of lines
#    cleaned = []
#    for line in lines:
#        for x1,y1,x2,y2 in line:
#            if abs(y2-y1) <=1 and abs(x2-x1) >=25 and abs(x2-x1) <= 55:
#                cleaned.append((x1,y1,x2,y2))
    
    #Step 2: Sort cleaned by x1 position
#    import operator
#    list1 = sorted(cleaned, key=operator.itemgetter(0, 1))
    
    #Step 3: Find clusters of x1 close together - clust_dist apart
#    clusters = {}
#    dIndex = 0
#    clus_dist = 10

#    for i in range(len(list1) - 1):
#        distance = abs(list1[i+1][0] - list1[i][0])
    #         print(distance)
#        if distance <= clus_dist:
#            if not dIndex in clusters.keys(): clusters[dIndex] = []
#            clusters[dIndex].append(list1[i])
#            clusters[dIndex].append(list1[i + 1])

#        else:
#            dIndex += 1
    
    #Step 4: Identify coordinates of rectangle around this cluster
#    rects = {}
#    i = 0
#    for key in clusters:
#        all_list = clusters[key]
#        cleaned = list(set(all_list))
#        if len(cleaned) > 5:
#            cleaned = sorted(cleaned, key=lambda tup: tup[1])
#            avg_y1 = cleaned[0][1]
#            avg_y2 = cleaned[-1][1]
    #         print(avg_y1, avg_y2)
#            avg_x1 = 0
#            avg_x2 = 0
#            for tup in cleaned:
#                avg_x1 += tup[0]
#               avg_x2 += tup[2]
#            avg_x1 = avg_x1/len(cleaned)
#            avg_x2 = avg_x2/len(cleaned)
#            rects[i] = (avg_x1, avg_y1, avg_x2, avg_y2)
#            i += 1
    
#    print("Num Parking Lanes: ", len(rects))
    #Step 5: Draw the rectangles on the image
#    buff = 7
#    for key in rects:
#        tup_topLeft = (int(rects[key][0] - buff), int(rects[key][1]))
#        tup_botRight = (int(rects[key][2] + buff), int(rects[key][3]))
#         print(tup_topLeft, tup_botRight)
#        cv2.rectangle(new_image, tup_topLeft,tup_botRight,(0,255,0),3)
#    return new_image, rects




# images showing the region of interest only
#rect_images = []
#rect_coords = []
#for image, lines in zip(test_images, list_of_lines):
#new_image, rects = identify_blocks(line_image, lines)
#rect_images.append(new_image)
#rect_coords.append(rects)
    
#show_images(rect_images)


#plt.imshow(new_image , cmap='gray')
#plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
#plt.show() 


#linhas_paralelas = findparallel(lines)
#lines[0]

#import itertools

#paralelas = ([])
#for pair in itertools.combinations(lines, r=3):
#    paralelas.append(([pair[0], pair[1]]))
    
#paralelas[0]
#len(paralelas)    

#imgTeste = np.copy(crop_img)

#cv2.rectangle(imgTeste,(384,300),(510,128),(0,255,0),3)

#plt.imshow(imgTeste , cmap='gray')
#plt.subplots_adjust(left=0.0, right=4.0, bottom=0.0, top=3.0) 
#plt.show() 

#https://stackoverflow.com/questions/15780210/python-opencv-detect-parallel-lines
#def findparallel(lines):
#    lines1 = []
#    for i in range(len(lines)-1):
#        for j in range(len(lines)-1):
#            if (i == j):continue
#            if (abs(lines[i][1] - lines[j][1]) == 0):          
#                 #You've found a parallel line!
#                     lines1.append((i,j))


#    return lines1